# Day 22: Reactor Reboot

In [17]:
from __future__ import annotations
from pathlib import Path
import re
from typing import Iterable
from enum import Enum, auto
from dataclasses import dataclass
from functools import reduce
from itertools import product
from more_itertools import collapse

from aoc2021.util import read_as_list

## Puzzle input data

In [ ]:
def parse_line(line: str) -> tuple[str,list[tuple[str]]]:
    action, coords = line.rstrip().split()
    coords = [tuple(map(int, re.findall('[\-\d]+', c))) for c in coords.split(',')]
    return action, coords

# Test data.
tdata0 = list(map(parse_line, [
    'on x=10..12,y=10..12,z=10..12',
    'on x=11..13,y=11..13,z=11..13',
    'off x=9..11,y=9..11,z=9..11',
    'on x=10..10,y=10..10,z=10..10',
]))
tdata1 = read_as_list(Path('./day22-test-input1.txt'), func=parse_line)
tdata2 = read_as_list(Path('./day22-test-input2.txt'), func=parse_line)

# Input data.
data = read_as_list(Path('./day22-input.txt'), func=parse_line)
data[:5]

## Puzzle answers
### Part 1

In [9]:
Input = list[tuple[str,list[tuple[str]]]]


class State(Enum):
    ON = auto()
    OFF = auto()


@dataclass(frozen=True)
class Pos:
    x: int
    y: int
    z: int


@dataclass(frozen=True)
class Cube:
    pos: Pos
    state: State


@dataclass(frozen=True)
class Space:
    xlim: tuple[int]
    ylim: tuple[int]
    zlim: tuple[int]

    @property
    def lims(self) -> Iterable[int]:
        return collapse((self.xlim, self.ylim, self.zlim))


@dataclass(frozen=True)
class Instr:
    action: State
    space: Space


def parse_instr(instr: tuple[str,list[tuple[str]]]) -> Instr:
    action = State(State.ON if instr[0] == 'on' else State.OFF)
    space = Space(*instr[1])
    return Instr(action, space)


def execute_instr(instr: Instr, cubes: set[Cube] = set()) -> set[Cube]:
    sp = instr.space
    new_cubes = set(Cube(Pos(*coords), State.ON) for coords in product(
        range(sp.xlim[0], sp.xlim[1]+1),
        range(sp.ylim[0], sp.ylim[1]+1),
        range(sp.zlim[0], sp.zlim[1]+1)
    ))
    return cubes | new_cubes if instr.action == State.ON else cubes - new_cubes


def execute_reboot(instrs: Input) -> Iterable[set[Cube]]:
    fn = lambda instr: all(abs(lim) <= 50 for lim in instr.space.lims)
    return reduce(lambda c,i: execute_instr(i,c), filter(fn, map(parse_instr, instrs)), set())


def solution(data: Input) -> int:
    cubes = execute_reboot(data)
    return sum(1 for c in cubes if c.state == State.ON)


assert parse_instr(('on', [(-49,-5),(-12,39),(-38,10)])) == Instr(State.ON, Space(xlim=(-49,-5), ylim=(-12,39), zlim=(-38,10)))
assert parse_instr(('off', [(18,30),(-20,-8),(-3,13)])) == Instr(State.OFF, Space(xlim=(18,30), ylim=(-20,-8), zlim=(-3,13)))
assert execute_instr(parse_instr(('on', [(-1,0),(2,2),(3,3)]))) == set([Cube(Pos(-1,2,3), State.ON), Cube(Pos(0,2,3), State.ON)])
assert execute_reboot([('on', [(-1,0),(2,2),(3,3)])]) == set([Cube(Pos(-1,2,3), State.ON), Cube(Pos(0,2,3), State.ON)])
assert solution(tdata0) == 39
assert solution(tdata1) == 590784

In [10]:
n = solution(data)
print(f'Number of cubes after reboot considering only the region x=-50..50,y=-50..50,z=-50..50: {n}')

Number of cubes after reboot considering only the region x=-50..50,y=-50..50,z=-50..50: 564654


### Part 2

In [34]:
from more_itertools import minmax

In [ ]:
@dataclass(frozen=True)
class Cuboid:
    xlim: tuple[int]
    ylim: tuple[int]
    zlim: tuple[int]

    @property
    def lims(self) -> Iterable[int]:
        return collapse((self.xlim, self.ylim, self.zlim))

    def __or__(self, other: Cuboid) -> Cuboid:
        # union
        xlim = minmax(self.xlim + other.xlim)
        ylim = minmax(self.ylim + other.ylim)
        zlim = minmax(self.zlim + other.zlim)
        return Cuboid(xlim, ylim, zlim)

    def __sub__(self, other: Cuboid) -> Cuboid:
        # difference
        xlim = (other.xlim[1] if self.xlim[0] < other.xlim[1] < self.xlim[1], )
        return Cuboid(xlim, ylim, zlim)


def range_diff(a: tuple[int], b: tuple[int]) -> tuple[int]:
    alo, aup = a
    blo, bup = b
    lower = alo 
    return


assert solution(tdata2) == 2758514936282235

In [ ]:
n = solution(data)
print(f'Number of cubes after reboot considering all cubes: {n}')